# Web scraping AirBnB

In [294]:
import os
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Get current working directory
print(os.getcwd())

/Users/andreas/Nextcloud/cloud_atrip/Dokumente/Repos/applied-data-science


# Url to scrape

In [295]:
# urlStringHome = 'https://www.airbnb.ch/'
urlString = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt"

In [296]:
# Set options for Chrome-Driver (headless mode)
opts = Options()
opts.add_argument("--window-size=1200,800")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Website to scrape
driver.get(urlString)

# Get current url
print(driver.current_url)

# Return the full page HTML code
# print(driver.page_source)

# Gets the page's title
print(driver.title)

# Close driver
# driver.close()

https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt
Erstklassige Unterkünfte & Ferienunterkünfte in Zürich: Gesamte Unterkunft | Airbnb - Airbnb


In [297]:
time.sleep(5)
# wait till cookie banner pops up
try:
    # close cookie banner
    cookie = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html[@class='js-focus-visible']/body[@class='with-new-header']/div[5]/div/div/div/div[1]/div[3]/section/div[2]/div[2]/button"))) 
    cookie.click()
except:
    pass

In [298]:
# get entries of apparements on page
nr_appartements = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div"))
nr_appartements

18

In [299]:
#### TODO SCRAP THIS STUFF ####
# implement scrapping logic in this func
def scrap_appartement(nr_of_flat):
    #### TODO SCRAP THIS STUFF ####
    print(nr_of_flat)
    ## check if element is present
    try:
        element_present = EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]"))
        WebDriverWait(driver, 2).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")

In [300]:
def handle_appartements_tabs(nr_appartements):
    # Click on each appartement which is opened in seperate tab
    for nr in range(1, nr_appartements):
        xpath = f"/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[{nr}]"
        driver.find_element(By.XPATH, xpath).click()

        try:
            # switch tab
            #get old tab
            parent = driver.window_handles[0]
            #get new tab
            chld = driver.window_handles[1]
            #switch to new tab
            driver.switch_to.window(chld)
        except:
            print("could not swtich tab")

        if nr == 1:
            try:
                # close translate banner
                transl = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[9]/section/div/div/div[2]/div/div[1]/button"))) 
                transl.click()
            except:
                pass
        
        scrap_appartement(nr)

        try: 
            # close tab
            parent = driver.window_handles[0]
            chld = driver.window_handles[1]
            driver.switch_to.window(chld)
            driver.close()
            time.sleep(0.5)
            parent = driver.window_handles[0]
            driver.switch_to.window(parent)
        except:
            print("could not close tab")
    

In [301]:
nr_pages = driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/a[4]").text
nr_pages = int(nr_pages)
nr_pages

15

In [302]:
for nr_p in range(1, nr_pages):
    handle_appartements_tabs(nr_appartements + 1)
    if nr_p == 1:
        driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/a[5]").click()
    else:
        driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/a[6]").click()

18
could not swtich tab
18
Timed out waiting for page to load
could not close tab
could not swtich tab
18
Timed out waiting for page to load
could not close tab


In [304]:
parent = driver.window_handles[0]
driver.switch_to.window(parent)
driver.close()